## Import pandas, numpy, sqlite3

In [49]:
import pandas as pd 
import numpy as np
import sqlite3
conn = sqlite3.connect('CourseDataWareHouse.db')
c = conn.cursor()

## Run this to set up %sql and %%sql magics and connect to CourseData Database

In [50]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [51]:
%%sql
sqlite:///CourseDataWareHouse.db

'Connected: @CourseDataWareHouse.db'

Drop Data Warehouse Tables For Rerun Ability

In [52]:
%%sql
DROP TABLE IF EXISTS COURSES;

 * sqlite:///CourseDataWareHouse.db
Done.


[]

In [53]:
%%sql
DROP TABLE IF EXISTS MEETINGS;

 * sqlite:///CourseDataWareHouse.db
Done.


[]

In [54]:
%%sql
DROP TABLE IF EXISTS PROFESSOR;

 * sqlite:///CourseDataWareHouse.db
Done.


[]

In [55]:
%%sql
DROP TABLE IF EXISTS TIME;

 * sqlite:///CourseDataWareHouse.db
Done.


[]

In [56]:
%%sql
DROP TABLE IF EXISTS COURSE_FACTS;

 * sqlite:///CourseDataWareHouse.db
Done.


[]

Create Data Warehouse Tables

In [57]:
%%sql
create table 'COURSES' (
    CC_ID INTEGER PRIMARY KEY,
    COURSE_TITLE VARCHAR NOT NULL, 
    PRE_REQS VARCHAR, 
    CO_REQS VARCHAR  
);

 * sqlite:///CourseDataWareHouse.db
Done.


[]

In [58]:
%%sql
create table 'MEETINGS' (
    ROOM_ID INTEGER PRIMARY KEY,
    ROOM_NO VARCHAR NOT NULL
);

 * sqlite:///CourseDataWareHouse.db
Done.


[]

In [59]:
%%sql
create table 'PROFESSOR'(
    I_ID INTEGER PRIMARY KEY,
    NAME VARCHAR NOT NULL,
    COURSE_TITLE VARCHAR NOT NULL
);

 * sqlite:///CourseDataWareHouse.db
Done.


[]

In [60]:
%%sql
create table 'TIME'(
    TIME_ID INTEGER PRIMARY KEY,
    START_TIME VARCHAR NOT NULL,
    END_TIME VARCHAR NOT NULL
);

 * sqlite:///CourseDataWareHouse.db
Done.


[]

In [61]:
%%sql
create table 'COURSE_FACTS'(
    CF_ID INTEGER PRIMARY KEY,
    ACT INTEGER,
    CAP INTEGER NOT NULL,
    REM INTEGER,
    AVG_CAP INTEGER NOT NULL,
    MAX_CAP INTEGER NOT NULL,
    MIN_CAP INTEGER,
    CC_ID INTEGER NOT NULL,
    ROOM_ID INTEGER,
    I_ID INTEGER NOT NULL,
    TIME_ID INTEGER,
    TERM_NAME VARCHAR NOT NULL,
    CRN INTEGER NOT NULL,
    FOREIGN KEY (CC_ID) REFERENCES COURSES (CC_ID)
    FOREIGN KEY (ROOM_ID) REFERENCES MEETINGS (ROOM_ID)
    FOREIGN KEY (I_ID) REFERENCES PROFESSOR (I_ID)
    FOREIGN KEY (TIME_ID) REFERENCES TIME (TIME_ID)
);

 * sqlite:///CourseDataWareHouse.db
Done.


[]

Insert Data into Data Warehouse Tables and select 2 rows as an eye check

In [62]:
%%sql
INSERT INTO COURSES (CC_ID, COURSE_TITLE, PRE_REQS, CO_REQS)
SELECT DISTINCT CC_ID, COURSE_TITLE, PRE_REQS, CO_REQS
FROM CATALOG_COURSE;

 * sqlite:///CourseDataWareHouse.db
2220 rows affected.


[]

In [63]:
%%sql
SELECT *
FROM COURSES
LIMIT 2;

 * sqlite:///CourseDataWareHouse.db
Done.


CC_ID,COURSE_TITLE,PRE_REQS,CO_REQS
1,Independent Study,None,None
2,Asian Studies Seminar,None,None


In [64]:
%%sql
INSERT INTO MEETINGS (ROOM_ID, ROOM_NO)
SELECT DISTINCT ROOM_ID, ROOM_NO
FROM LOCATION;

 * sqlite:///CourseDataWareHouse.db
13032 rows affected.


[]

In [65]:
%%sql
SELECT *
FROM MEETINGS
LIMIT 2;

 * sqlite:///CourseDataWareHouse.db
Done.


ROOM_ID,ROOM_NO
1,DSB 105
2,DSB 105


In [66]:
%%sql
INSERT INTO PROFESSOR (I_ID, NAME, COURSE_TITLE)
SELECT DISTINCT I_ID, NAME , COURSE_TITLE
FROM INSTRUCTOR;

 * sqlite:///CourseDataWareHouse.db
14330 rows affected.


[]

In [67]:
%%sql
SELECT *
FROM PROFESSOR
LIMIT 2;

 * sqlite:///CourseDataWareHouse.db
Done.


I_ID,NAME,COURSE_TITLE
1,Michael P. Coyne,Introduction to Financial Accounting
2,Michael P. Coyne,Introduction to Financial Accounting


In [68]:
%%sql
INSERT INTO TIME (time_id, start_time, end_time)
SELECT DISTINCT time_id, start_time, end_time
FROM TIME_CODE;

 * sqlite:///CourseDataWareHouse.db
296781 rows affected.


[]

In [69]:
%%sql
SELECT *
FROM TIME
LIMIT 2;

 * sqlite:///CourseDataWareHouse.db
Done.


TIME_ID,START_TIME,END_TIME
1,2014-09-02T08:00:00,2014-09-02T09:15:00
2,2014-09-05T08:00:00,2014-09-05T09:15:00


In [70]:
%%sql
INSERT INTO COURSE_FACTS (ACT, CAP, REM, AVG_CAP, MAX_CAP, MIN_CAP, CC_ID, 
ROOM_ID, I_ID, TIME_ID, TERM_NAME, CRN)
select distinct ACT, CAP, REM, 
avg(cap), max(cap), min(cap), 
CC_ID, ROOM_ID, I_ID, TIME_ID, a.TERM_NAME, a.crn
from COURSE_MEETING a
    inner join course_offering b on a.crn = b.crn and a.term_name = b.term_name
    LEFT JOIN CATALOG_COURSE c on b.course_title = c.course_title 
    LEFT JOIN LOCATION d on a.crn = d.crn and a.term_name = d.term_name
    LEFT JOIN INSTRUCTOR e on b.course_title = e.course_title and a.term_name = e.term_name
    LEFT JOIN TIME_CODE f on a.crn = f.crn and a.term_name = f.term_name
group by ACT, CAP, REM, CC_ID, ROOM_ID, I_ID, TIME_ID, a.TERM_NAME, a.crn
;

 * sqlite:///CourseDataWareHouse.db
2899784 rows affected.


[]

In [71]:
%%sql
select *
from COURSE_FACTS
limit 2;

 * sqlite:///CourseDataWareHouse.db
Done.


CF_ID,ACT,CAP,REM,AVG_CAP,MAX_CAP,MIN_CAP,CC_ID,ROOM_ID,I_ID,TIME_ID,TERM_NAME,CRN
1,-1,1,0,1,1,1,774,None,783,None,Fall2014,70848
2,-1,1,0,1,1,1,775,None,783,None,Fall2014,70848


Drop the fact table because its too big to push to github. Will recreate it in code.

In [72]:
%%sql
DROP TABLE IF EXISTS COURSE_FACTS;

 * sqlite:///CourseDataWareHouse.db
Done.


[]

Vacuum to tidy up

In [73]:
%%sql
VACUUM

 * sqlite:///CourseDataWareHouse.db
Done.


[]